In [5]:
import pandas as pd
import numpy as np

Conexion hacia el SNOWFLAKE

In [6]:
#snowflake connection
import snowflake.connector
import os

conn=snowflake.connector.connect(

    user=os.getenv('SNOWFLAKE_USERNAME'),
    password=os.getenv('SNOWFLAKE_PASSWORD'),
    account='zsb67146.us-east-1',
    warehouse='COMPUTE_WH',
    database='PROYECTO1',
    schema='RAW'

)

Guardado de tablas

In [7]:
list_tables_query = 'SHOW TABLES'
tables = pd.read_sql(list_tables_query, conn)
print(tables)

C:\Users\elian\AppData\Local\Temp\ipykernel_11136\1362887654.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables = pd.read_sql(list_tables_query, conn)


                        created_on         name database_name schema_name  \
0 2025-02-12 06:01:43.602000+00:00       AISLES     PROYECTO1         RAW   
1 2025-02-12 06:01:49.021000+00:00  DEPARTMENTS     PROYECTO1         RAW   
2 2025-02-12 06:02:04.210000+00:00    INSTACART     PROYECTO1         RAW   
3 2025-02-12 06:02:28.195000+00:00       ORDERS     PROYECTO1         RAW   
4 2025-02-12 06:02:44.855000+00:00     PRODUCTS     PROYECTO1         RAW   

    kind comment cluster_by     rows     bytes         owner  ...  \
0  TABLE                         134      3072  ACCOUNTADMIN  ...   
1  TABLE                          21      1536  ACCOUNTADMIN  ...   
2  TABLE                      478967   3629056  ACCOUNTADMIN  ...   
3  TABLE                     4545007  25421824  ACCOUNTADMIN  ...   
4  TABLE                       49694   1080832  ACCOUNTADMIN  ...   

  search_optimization_bytes is_external enable_schema_evolution  \
0                      None           N                

In [8]:

conn.cursor().execute("USE DATABASE PROYECTO1")
conn.cursor().execute("USE SCHEMA RAW")

In [9]:
#query to get the data from the table

names= ["AISLES","DEPARTMENTS","INSTACART","ORDERS","PRODUCTS"]

tables={}

for name in names:
   query= f"select * from {name}"
   tables[name]=pd.read_sql(query,conn)

C:\Users\elian\AppData\Local\Temp\ipykernel_11136\2803079868.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables[name]=pd.read_sql(query,conn)


In [10]:
for name in names:
    print(f"Shape of {name} table is {tables[name].shape}")

Shape of AISLES table is (134, 2)
Shape of DEPARTMENTS table is (21, 2)
Shape of INSTACART table is (478967, 6)
Shape of ORDERS table is (4545007, 4)
Shape of PRODUCTS table is (49694, 5)


Valores repetidos en 

In [11]:
tableFrequency={}

for name in names:
    print(name)
    tableFrequency[name] = tables[name].iloc[:, 0].value_counts()
    tableFrequencyofFrequency = tableFrequency[name].value_counts()
    
    # Filter frequencies where the count is two or greater
    filteredFrequency = tableFrequencyofFrequency[tableFrequencyofFrequency.index >1]
    
    print(filteredFrequency)
    print("\n")

AISLES
Series([], Name: count, dtype: int64)


DEPARTMENTS
Series([], Name: count, dtype: int64)


INSTACART
count
2    15
Name: count, dtype: int64


ORDERS
count
5      31923
6      31698
4      31054
7      30822
3      29046
       ...  
87         1
85         1
80         1
72         1
127        1
Name: count, Length: 89, dtype: int64


PRODUCTS
Series([], Name: count, dtype: int64)




We remove all the duplicate values in the tables

In [12]:
# drop duplicates on all tables
for name in names:
    tables[name].drop_duplicates(inplace=True)

In [13]:
tableFrequency={}

for name in names:
    print(name)
    tableFrequency[name] = tables[name].iloc[:, 0].value_counts()
    tableFrequencyofFrequency = tableFrequency[name].value_counts()
    
    # Filter frequencies where the count is two or greater
    filteredFrequency = tableFrequencyofFrequency[tableFrequencyofFrequency.index >1]
    
    print(filteredFrequency)
    print("\n")

AISLES
Series([], Name: count, dtype: int64)


DEPARTMENTS
Series([], Name: count, dtype: int64)


INSTACART
Series([], Name: count, dtype: int64)


ORDERS
count
5      31923
6      31698
4      31054
7      30822
3      29046
       ...  
87         1
85         1
80         1
72         1
127        1
Name: count, Length: 89, dtype: int64


PRODUCTS
Series([], Name: count, dtype: int64)




Solamente habian duplicados en INSTACART la frequencia de las otras ordenes se repite 

Now that we have dropped the duplicates we will be demonstrating the data for each characteristics of the table

DATA INFO

AISLES TABLE

In [14]:
#describe
tables["AISLES"].describe()


,aisle_id
count,134.000000
mean,67.500000
std,38.826537
min,1.000000
25%,34.250000
50%,67.500000
75%,100.750000
max,134.000000


DEPARTMENTS TABLE

In [15]:
tables["DEPARTMENTS"].describe()

,department_id
count,21.000000
mean,11.000000
std,6.204837
min,1.000000
25%,6.000000
50%,11.000000
75%,16.000000
max,21.000000


PRODUCTS TABLE

In [16]:
tables["PRODUCTS"].describe()

,product_id,aisle_id,department_id
count,49694.000000,49694.000000,49694.000000
mean,24847.500000,67.773192,11.728478
std,14345.566475,38.317265,5.850314
min,1.000000,1.000000,1.000000
25%,12424.250000,35.000000,7.000000
50%,24847.500000,69.000000,13.000000
75%,37270.750000,100.000000,17.000000
max,49694.000000,134.000000,21.000000


INSTACART_ORDERS TABLE

In [17]:
tables["INSTACART"][["order_number", "order_dow", "order_hour_of_day", "days_since_prior_order"]].describe()

,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,478952.000000,478952.000000,478952.000000,450135.000000
mean,17.157563,2.775051,13.447034,11.101814
std,17.704457,2.045901,4.224567,9.190004
min,1.000000,0.000000,0.000000,0.000000
25%,5.000000,1.000000,10.000000,4.000000
50%,11.000000,3.000000,13.000000,7.000000
75%,23.000000,5.000000,16.000000,15.000000
max,100.000000,6.000000,23.000000,30.000000


ORDERS TABLE 

In [18]:
tables["ORDERS"].describe().drop("count")

,order_id,product_id,add_to_cart_order,reordered
mean,1.711166e+06,25580.835682,8.351669,0.590503
std,9.850955e+05,14095.518725,7.080368,0.491741
min,4.000000e+00,1.000000,1.000000,0.000000
25%,8.608170e+05,13535.000000,3.000000,0.000000
50%,1.711530e+06,25272.000000,6.000000,1.000000
75%,2.561311e+06,37935.000000,11.000000,1.000000
max,3.421079e+06,49694.000000,64.000000,1.000000


In [19]:
#only get reordered column
print("Reordered Column in Orders Table (0,1)")
tables["ORDERS"]["reordered"].describe().drop(['count', '25%', '50%', '75%', 'max', 'min'])

Reordered Column in Orders Table (0,1)


mean    0.590503
std     0.491741
Name: reordered, dtype: float64

Eliminacion de valores nulos ("", Nan, NULL)

In [20]:
missing_values = tables["ORDERS"][tables["ORDERS"]['add_to_cart_order'].isna()]

missing_values

,order_id,product_id,add_to_cart_order,reordered
737,2449164,5068,NaN,0
9926,1968313,43867,NaN,0
14394,2926893,11688,NaN,0
16418,1717990,4142,NaN,0
30114,1959075,42828,NaN,1
...,...,...,...,...
4505662,1800005,7411,NaN,0
4511400,1633337,260,NaN,0
4517562,404157,9517,NaN,0
4534112,1673227,17835,NaN,0


In [21]:
##Missing (NaN) values in ORDERS table

missing_values = tables["ORDERS"][tables["ORDERS"]['add_to_cart_order'].isna()]

nan_count = tables["ORDERS"]['add_to_cart_order'].isna().sum()

nan_count

836

Ya que hay 836 valores en la columan de 'add_to_cart_order', cuando en total hay mas de 4 millones,se van a eliminar estas columnas ya que no tienen tanta influencia tomando que es un data set muy grande.

In [22]:
tables["ORDERS"]=tables["ORDERS"][tables["ORDERS"]['add_to_cart_order'].notna()]
missing_values = tables["ORDERS"][tables["ORDERS"]['add_to_cart_order'].isna()]
nan_count = tables["ORDERS"]['add_to_cart_order'].isna().sum()
nan_count

0

Ahora se va a eliminar los valures nulos en INSTACART , existen valores nulos en "days_since_prior_order" en donde sea la primera vez que el usuario orden algo.

Vamos a convertir en 0 los NaN values y reemplazarlos en estas filas

In [23]:
missing_values = tables["INSTACART"][tables["INSTACART"]['days_since_prior_order'].isna()]

missing_values


,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
28,133707,182261,1,3,10,NaN
96,787445,25685,1,6,18,NaN
100,294410,111449,1,0,19,NaN
103,2869915,123958,1,4,16,NaN
104,2521921,42286,1,3,18,NaN
...,...,...,...,...,...,...
478895,2589657,205028,1,0,16,NaN
478896,2222353,141211,1,2,13,NaN
478922,2272807,204154,1,1,15,NaN
478926,2499542,68810,1,4,19,NaN


In [24]:
temp=tables["INSTACART"][tables["INSTACART"]["order_number"]<2]
temp

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
28,133707,182261,1,3,10,NaN
96,787445,25685,1,6,18,NaN
100,294410,111449,1,0,19,NaN
103,2869915,123958,1,4,16,NaN
104,2521921,42286,1,3,18,NaN
...,...,...,...,...,...,...
478895,2589657,205028,1,0,16,NaN
478896,2222353,141211,1,2,13,NaN
478922,2272807,204154,1,1,15,NaN
478926,2499542,68810,1,4,19,NaN


In [25]:
tables["INSTACART"].loc[tables["INSTACART"]["order_number"]<2,"days_since_prior_order"]=0


In [26]:
temp=tables["INSTACART"][tables["INSTACART"]["order_number"]<2]
temp

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
28,133707,182261,1,3,10,0.0
96,787445,25685,1,6,18,0.0
100,294410,111449,1,0,19,0.0
103,2869915,123958,1,4,16,0.0
104,2521921,42286,1,3,18,0.0
...,...,...,...,...,...,...
478895,2589657,205028,1,0,16,0.0
478896,2222353,141211,1,2,13,0.0
478922,2272807,204154,1,1,15,0.0
478926,2499542,68810,1,4,19,0.0


In [27]:
tables["INSTACART"]["order_number"]=tables["INSTACART"]["order_number"].astype(int)

In [28]:
productSize=tables["PRODUCTS"].size

print(f"Products Table Size: {productSize}")



Products Table Size: 248470


Dentro de la tabla de productos tenemos nombres de productos que no existen ,juntamente con su precio. Por lo cual vamos a eliminarlos igualmente

In [29]:
tables["PRODUCTS"].head()

,product_id,product_name,aisle_id,department_id,price
0,1,Chocolate Sandwich Cookies,61,19,None
1,2,All-Seasons Salt,104,13,None
2,3,Robust Golden Unsweetened Oolong Tea,94,7,None
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,None
4,5,Green Chile Anytime Sauce,5,13,None


In [30]:
tables["PRODUCTS"].drop(columns=["price"], inplace=True)

In [31]:
tables["PRODUCTS"].head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [32]:
tables["PRODUCTS"][tables["PRODUCTS"]["product_name"].isnull()]

,product_id,product_name,aisle_id,department_id


In [33]:
#Convertir a NaN verdadero

tables["PRODUCTS"]["product_name"].replace(["", "NaN"], pd.NA, inplace=True)

In [34]:
tables["PRODUCTS"][tables["PRODUCTS"]["product_name"].isnull()]

,product_id,product_name,aisle_id,department_id
37,38,NaN,100,21
71,72,NaN,100,21
109,110,NaN,100,21
296,297,NaN,100,21
416,417,NaN,100,21
...,...,...,...,...
49552,49553,NaN,100,21
49574,49575,NaN,100,21
49640,49641,NaN,100,21
49663,49664,NaN,100,21


In [35]:
tables["PRODUCTS"]=tables["PRODUCTS"].dropna()

In [36]:
tables["PRODUCTS"][tables["PRODUCTS"]["product_name"].isnull()]

,product_id,product_name,aisle_id,department_id


In [37]:
tables["PRODUCTS"].head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [38]:
for name in tables:
    print(name)

AISLES
DEPARTMENTS
INSTACART
ORDERS
PRODUCTS


Transferencia de datos a Star Schema

In [39]:
factOrder=tables["ORDERS"]
factInstaCart=tables["INSTACART"]

productsDim=tables["PRODUCTS"]

productsDim.drop(columns=["aisle_id","department_id"], inplace=True)

In [40]:
aislesNames=tables["AISLES"]["aisle"]
productsDim["aisle_name"]=aislesNames

departmentNames=tables["DEPARTMENTS"]["department"]
productsDim["department_name"]=departmentNames

In [42]:
productsDim.head()

,product_id,product_name,aisle_name,department_name
0,1,Chocolate Sandwich Cookies,prepared soups salads,frozen
1,2,All-Seasons Salt,specialty cheeses,other
2,3,Robust Golden Unsweetened Oolong Tea,energy granola bars,bakery
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,instant foods,produce
4,5,Green Chile Anytime Sauce,marinades meat preparation,alcohol
